In [2]:
# In[0]: IMPORT AND FUNCTIONS
import os
import warnings
from datetime import timedelta

import joblib
import numpy as np
import optuna
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import randint, uniform, loguniform
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.exceptions import ConvergenceWarning
from sklearn.impute import SimpleImputer
from sklearn.linear_model import BayesianRidge, Ridge, LinearRegression, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import (KFold, RandomizedSearchCV, cross_val_predict,
                                     cross_val_score, train_test_split)
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, StandardScaler
from sklearn.svm import SVR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# Suppress specific warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", message="A value is trying to be set on a copy of a DataFrame")
warnings.filterwarnings("ignore", message="No further splits with positive gain")
warnings.filterwarnings("ignore", category=ConvergenceWarning)


# Create directories to save figures, models, and saved objects
os.makedirs('figures', exist_ok=True)
os.makedirs('models', exist_ok=True)
os.makedirs('saved_objects', exist_ok=True)

# KFold split configuration
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Function to store models
def store_model(model, model_name=""):
    """Store the trained model to the 'models' folder."""
    if model_name == "":
        model_name = type(model).__name__
    joblib.dump(model, f'models/{model_name}_model.pkl')
    print(f'Model {model_name} saved successfully.')

# Function to load models
def load_model(model_name):
    """Load a model from the 'models' folder."""
    model = joblib.load(f'models/{model_name}_model.pkl')
    print(f'Model {model_name} loaded successfully.')
    return model

# In[1]: LOAD DATA AND INITIAL PREPROCESSING
raw_data = pd.read_csv('datasets/NewYork.csv')
raw_data['datetime'] = pd.to_datetime(raw_data['datetime'])
raw_data.drop(columns=["name", "icon", "stations", "description"], inplace=True)

print("\nMissing values before imputation:")
print(raw_data.isnull().sum())

# Outlier removal function
def remove_outliers(df, column, factor=1.5):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - factor * IQR
    upper_bound = Q3 + factor * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

raw_data = remove_outliers(raw_data, 'tempmax')
print("\nShape of data after removing outliers:", raw_data.shape)

# Handle missing values using proper assignments to avoid chained assignment warning
for column in raw_data.columns:
    if raw_data[column].dtype == 'object':
        raw_data[column] = raw_data[column].fillna('Unknown')
    else:
        raw_data[column] = raw_data[column].fillna(raw_data[column].median())

print("\nMissing values after imputation:")
print(raw_data.isnull().sum())

# In[2]: DISCOVER THE DATA
print('\n____________ Dataset info ____________')
print(raw_data.info())              
print('\n____________ Some first data examples ____________')
print(raw_data.head(3)) 
print('\n____________ Statistics of numeric features ____________')
print(raw_data.describe())    

# Correlation matrix and clustermap for numeric features
plt.figure(figsize=(12, 10))
numeric_data = raw_data.select_dtypes(include=[np.number])
corr = numeric_data.corr()

# Handle any non-finite values in the correlation matrix
if not np.isfinite(corr.values).all():
    print("Warning: Correlation matrix contains non-finite values. Replacing with 0.")
    corr_array = np.nan_to_num(corr.values, nan=0, posinf=0, neginf=0)
    corr = pd.DataFrame(corr_array, index=corr.index, columns=corr.columns)

# Create the clustermap visualization
try:
    sns.clustermap(corr, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
    plt.title("Correlation Clustermap")
    plt.savefig('figures/correlation_clustermap.png', format='png', dpi=300)
except Exception as e:
    print(f"Error creating clustermap: {e}")
    print("Skipping clustermap creation.")
finally:
    plt.close()

# Histogram of all numeric features
numeric_features = raw_data.select_dtypes(include=[np.number]).columns
n_features = len(numeric_features)
n_rows = (n_features + 1) // 2
plt.figure(figsize=(15, 5 * n_rows))
for i, feature in enumerate(numeric_features, 1):
    plt.subplot(n_rows, 2, i)
    sns.histplot(raw_data[feature], kde=True)
    plt.title(f'Distribution of {feature}')
plt.tight_layout()
plt.savefig('figures/hist_raw_data.png', format='png', dpi=300)
plt.close()

# Scatter plots of tempmax vs other numeric features
numeric_features = [col for col in numeric_features if col != 'tempmax']
n_features = len(numeric_features)
n_rows = (n_features + 1) // 2
plt.figure(figsize=(15, 5 * n_rows))
for i, feature in enumerate(numeric_features, 1):
    plt.subplot(n_rows, 2, i)
    sns.scatterplot(data=raw_data, x=feature, y='tempmax', alpha=0.5)
    plt.title(f'Max Temperature vs {feature}')
plt.tight_layout()
plt.savefig('figures/scatter_tempmax_vs_features.png', format='png', dpi=300)
plt.close()

# Pairplot of main features
main_features = ['tempmax', 'temp', 'humidity', 'windspeed', 'cloudcover', 'tempmin', 'feelslikemax', 'feelslikemin', 'feelslike']
plt.figure(figsize=(15, 15))
sns.pairplot(raw_data[main_features], diag_kind='kde')
plt.suptitle("Pairplot of Main Features", y=1.02)
plt.savefig('figures/pairplot_main_features.png', format='png', dpi=300)
plt.close()

# In[3]: PREPARE THE DATA 
class EnhancedFeatureAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_features=True):
        self.add_features = add_features
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_ = X.copy()

        # Ensure that the input is a DataFrame before adding features
        if isinstance(X_, pd.DataFrame) and self.add_features and 'datetime' in X_.columns:
            # Add new features from datetime
            X_['day_of_year'] = X_['datetime'].dt.dayofyear
            X_['month'] = X_['datetime'].dt.month
            X_['day_of_week'] = X_['datetime'].dt.dayofweek
            X_['is_weekend'] = X_['day_of_week'].isin([5, 6]).astype(int)
            X_['day_of_year_sin'] = np.sin(2 * np.pi * X_['day_of_year'] / 365.25)
            X_['day_of_year_cos'] = np.cos(2 * np.pi * X_['day_of_year'] / 365.25)
            X_.drop(columns=['datetime'], inplace=True)
        return X_

# Function to add lag features
def add_lag_features(df, column, lags):
    for lag in lags:
        df[f'{column}_lag_{lag}'] = df[column].shift(lag)
    return df

# Function to add rolling averages
def add_rolling_features(df, column, windows):
    for window in windows:
        df[f'{column}_rolling_{window}'] = df[column].rolling(window).mean()
    return df

# Apply lag features and rolling averages to the selected columns
selected_columns = ['tempmax', 'tempmin', 'temp', 'feelslikemax', 'feelslikemin', 'feelslike']
for col in selected_columns:
    raw_data = add_lag_features(raw_data, col, lags=[1, 2, 3])
    raw_data = add_rolling_features(raw_data, col, windows=[3, 7])

# Evaluate Feature Correlation
numeric_data = raw_data.select_dtypes(include=[np.number])
correlation_matrix = numeric_data.corr()
tempmax_correlation = correlation_matrix['tempmax'].sort_values(ascending=False)
print("Correlation of features with tempmax:\n", tempmax_correlation)

# Select features with high correlation (e.g., correlation > 0.1 or < -0.1)
selected_features = tempmax_correlation[abs(tempmax_correlation) > 0.1].index.tolist()
selected_features.remove('tempmax')  # Remove target variable from features
print("Selected features based on correlation:\n", selected_features)

# Separate numeric and categorical features
numeric_features = [feature for feature in selected_features if raw_data[feature].dtype in [np.float64, np.int64]]
categorical_features = [feature for feature in selected_features if raw_data[feature].dtype == 'object']

# Define transformers for numeric and categorical features
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Define a preprocessor that applies transformations to numeric and categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Full pipeline with the EnhancedFeatureAdder and preprocessor
enhanced_pipeline = Pipeline(steps=[
    ('feature_adder', EnhancedFeatureAdder()),  # Ensure features are added here
    ('preprocessor', preprocessor)              # Apply transformations to the features
])

# Split features (X) and target (y)
X = raw_data.drop('tempmax', axis=1)
y = raw_data['tempmax']

# Apply the pipeline to the data
X_processed = enhanced_pipeline.fit_transform(X, y)

# Split the processed data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

# Models to be evaluated
models = {
    'RandomForestReg': RandomForestRegressor(random_state=42),
    'GradientBoostingReg': GradientBoostingRegressor(random_state=42),
    'LGBMReg': LGBMRegressor(random_state=42, verbosity=-1),
    'XGBBoost': XGBRegressor(random_state=42),
    'BayesianRidge': BayesianRidge(),
    'LinearReg': LinearRegression(),
    'Ridge': Ridge(random_state=42),
    'Lasso': Lasso(random_state=42),
    'SVR': SVR(),
    'MLPRegressor': MLPRegressor(random_state=42),
    'PolynomialReg': make_pipeline(PolynomialFeatures(), LinearRegression())
}

best_model_name = None
best_cross_val_rmse = float('inf')

print('\n____________ K-Fold Cross Validation and Residual Analysis ____________')
for name, model in models.items():
    # Create a pipeline for each model
    model_pipeline = Pipeline(steps=[('model', model)])
    
    # Perform K-Fold Cross-validation
    cv_rmse_scores = -cross_val_score(model_pipeline, X_train, y_train, cv=kfold, scoring='neg_mean_squared_error')
    avg_rmse = np.sqrt(cv_rmse_scores.mean())
    
    # Save the fold RMSE scores to saved_objects
    joblib.dump(cv_rmse_scores, f'saved_objects/{name}_rmse.pkl')
    
    print(f'{name:<20} K-Fold Avg RMSE: {avg_rmse:.4f}')
    
    # Calculate residuals
    y_train_pred = cross_val_predict(model_pipeline, X_train, y_train, cv=kfold)
    residuals = y_train - y_train_pred
    
    # Plot residuals
    plt.figure(figsize=(10, 6))
    sns.histplot(residuals, kde=True, bins=50)
    plt.title(f'Residuals Distribution - {name}')
    plt.savefig(f'figures/residuals_{name}.png', format='png', dpi=300)
    plt.close()
    
    # Store the best model based on RMSE
    if avg_rmse < best_cross_val_rmse:
        best_cross_val_rmse = avg_rmse
        best_model_name = name

# After K-Fold Cross-Validation and residual analysis, print the final model chosen
print(f"\nBest model after K-Fold Cross Validation: {best_model_name} with RMSE: {best_cross_val_rmse:.4f}")

# Assign the best model after cross-validation
best_model = models[best_model_name]

# Store the best model after cross-validation
store_model(best_model, model_name=best_model_name)

# In[5]: FINE-TUNE THE BEST MODEL ONLY
print(f'\n____________ Fine-tuning the best model: {best_model_name} ____________')

warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Ensure input data is scaled
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

param_grids = {
    'RandomForestReg': {
        'n_estimators': randint(100, 2000),
        'max_depth': randint(5, 50),
        'min_samples_split': randint(2, 20),
        'min_samples_leaf': randint(1, 20),
        'max_features': uniform(0.1, 0.9)
    },
    'GradientBoostingReg': {
        'n_estimators': randint(100, 2000),
        'learning_rate': uniform(0.01, 0.2),
        'max_depth': randint(3, 20),
        'min_samples_split': randint(2, 20),
        'min_samples_leaf': randint(1, 20),
        'subsample': uniform(0.5, 0.5)
    },
    'LGBMReg': {
        'num_leaves': randint(20, 200),
        'learning_rate': uniform(0.01, 0.2),
        'n_estimators': randint(100, 2000),
        'min_child_samples': randint(1, 50),
        'subsample': uniform(0.5, 0.5),
        'colsample_bytree': uniform(0.5, 0.5)
    },
    'XGBBoost': {
        'n_estimators': randint(100, 2000),
        'learning_rate': uniform(0.01, 0.2),
        'max_depth': randint(3, 20),
        'min_child_weight': randint(1, 10),
        'subsample': uniform(0.5, 0.5),
        'colsample_bytree': uniform(0.5, 0.5),
        'gamma': uniform(0, 0.5)
    },
    'BayesianRidge': {
        'alpha_1': loguniform(1e-6, 1e-1),
        'alpha_2': loguniform(1e-6, 1e-1),
        'lambda_1': loguniform(1e-6, 1e-1),
        'lambda_2': loguniform(1e-6, 1e-1)
    },
    'Ridge': {
        'alpha': loguniform(1e-3, 1e3),
        'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
    },
    'Lasso': {
        'alpha': loguniform(1e-3, 1e3)
    },
    'SVR': {
        'C': loguniform(1e-3, 1e3),
        'epsilon': loguniform(1e-3, 1e1),
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid']
    },
    'MLPRegressor': {
        'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
        'activation': ['tanh', 'relu'],
        'solver': ['sgd', 'adam'],
        'alpha': loguniform(1e-4, 1e-1),
        'learning_rate': ['constant','adaptive']
    },
    'PolynomialReg': {
        'polynomialfeatures__degree': randint(2, 5),
        'linearregression__fit_intercept': [True, False]
    }
}

# Fine-tune the best model using RandomizedSearchCV
grid_search = RandomizedSearchCV(best_model, param_distributions=param_grids.get(best_model_name, {}),
                                 n_iter=100, cv=kfold, 
                                 scoring='neg_mean_squared_error', n_jobs=-1, 
                                 random_state=42, verbose=1, error_score='raise')

grid_search.fit(X_train, y_train)

# Save the tuned model as 'SOLUTION_model.pkl'
best_model_tuned = grid_search.best_estimator_
store_model(best_model_tuned, model_name='SOLUTION_model')

# Save RMSE of the tuned model
best_tuned_rmse = np.sqrt(-grid_search.best_score_)
joblib.dump(best_tuned_rmse, 'saved_objects/SOLUTION_model_rmse.pkl')
print(f"Best RMSE for {best_model_name} after tuning: {best_tuned_rmse:.4f}")

# In[6]: ANALYZE AND TEST THE SOLUTION
print(f"\nBest RMSE for {best_model_name} after tuning: {np.sqrt(-grid_search.best_score_):.4f}")

# Make predictions on the test set
y_pred = best_model_tuned.predict(X_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'\nPerformance on test data: RMSE: {test_rmse:.4f}')

# Save test RMSE results
joblib.dump(test_rmse, 'saved_objects/test_rmse.pkl')

# In[8]: MAKE FUTURE PREDICTIONS

# Define the find_closest_date function
def find_closest_date(target_date, data):
    try:
        target_date = target_date.replace(year=target_date.year - 1)
    except ValueError:
        target_date = target_date.replace(year=target_date.year - 1, day=28)
    closest_date = data['datetime'].iloc[(data['datetime'] - target_date).abs().argsort()[0]]
    return data.loc[data['datetime'] == closest_date].iloc[0]

# Continue with the future predictions
last_date = raw_data['datetime'].max()
future_dates = pd.date_range(start=last_date + timedelta(days=1), periods=200)
future_data = pd.DataFrame({'datetime': future_dates})

for col in raw_data.columns:
    if col not in ['datetime', 'tempmax']:
        future_data[col] = future_data['datetime'].apply(lambda x: find_closest_date(x, raw_data)[col])

future_processed = enhanced_pipeline.transform(future_data)
future_pred = best_model_tuned.predict(future_processed)
future_data['predicted_tempmax'] = future_pred

# Save future predictions
future_data[['datetime', 'predicted_tempmax']].to_csv('future_predictions_200days.csv', index=False)

# Static visualization of future predictions
plt.figure(figsize=(20, 10))
plt.plot(future_data['datetime'], future_data['predicted_tempmax'], label='Predicted Max Temperature', alpha=0.7)
plt.title('Predicted Maximum Temperature for the Next 200 Days')

plt.xlabel('Date')
plt.ylabel('Temperature (°C)')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()

# Save the future prediction plot
plt.savefig('figures/future_predictions_200days_plot.png', format='png', dpi=300)
plt.close()

print("\nPredicted Max Temperature:")
print(f"Min: {future_data['predicted_tempmax'].min():.2f}°C")
print(f"Max: {future_data['predicted_tempmax'].max():.2f}°C")
print(f"Avg: {future_data['predicted_tempmax'].mean():.2f}°C")

# In[9]: CONCLUSION
print("\n____________ CONCLUSION ____________")
print(f"""
1. Data Preprocessing:
   - Removed outliers and handled missing values.
   - Added engineered features: day of year, month, day of week, is_weekend.
   - Applied scaling and one-hot encoding.

2. Model Selection and Hyperparameter Tuning:
   - Evaluated multiple models using RMSE as the sole metric.
   - Used RandomizedSearchCV for hyperparameter optimization.
   - The best performing model was: {best_model_name}

3. Model Performance:
   - Best model RMSE on test data: {test_rmse:.4f}

4. Future Predictions:
   - Generated predictions for the next 200 days.
   - The predicted temperatures range from {future_data['predicted_tempmax'].min():.2f}°C to {future_data['predicted_tempmax'].max():.2f}°C.

Suggestions for Further Improvement:
1. Collect more historical data or external data sources.
2. Experiment with more advanced time series models.
3. Implement online learning for continuous model updates.
4. Consider using deep learning models for complex pattern capture.
5. Analyze prediction intervals for more robust forecasts.
""")

print("\nPrediction and analysis complete.")


Missing values before imputation:
datetime              0
tempmax               0
tempmin               0
temp                  0
feelslikemax          0
feelslikemin          0
feelslike             0
dew                   0
humidity              0
precip                0
precipprob            0
precipcover           0
preciptype          386
snow                  0
snowdepth             0
windgust              0
windspeed             0
winddir               0
sealevelpressure      0
cloudcover            0
visibility            0
solarradiation        0
solarenergy           0
uvindex               0
severerisk           40
sunrise               0
sunset                0
moonphase             0
conditions            0
dtype: int64

Shape of data after removing outliers: (1000, 29)

Missing values after imputation:
datetime            0
tempmax             0
tempmin             0
temp                0
feelslikemax        0
feelslikemin        0
feelslike           0
dew              

<Figure size 1200x1000 with 0 Axes>

<Figure size 1500x1500 with 0 Axes>